In [6]:
import pandas as pd

# 1. 파일 불러오기
file_path = "(개방용) 지역본부별 시간대별 전력사용량_20241231.csv"
df = pd.read_csv(file_path, encoding='cp949')

# 2. 컬럼/공백 정리
df.columns = df.columns.str.strip()
df['본부명'] = df['본부명'].str.strip()

# 3. 날짜/시간 정리
df['기준일자'] = pd.to_datetime(df['기준일자'], errors='coerce')
df['기준시'] = pd.to_numeric(df['기준시'], errors='coerce')

# 4. 남서울본부만
south_df = df[df['본부명'] == '남서울본부'].copy()
south_df = south_df.sort_values(['기준일자', '기준시']).reset_index(drop=True)

# 5. 서울본부만
seoul_df = df[df['본부명'] == '서울본부'].copy()
seoul_df = seoul_df.sort_values(['기준일자', '기준시']).reset_index(drop=True)

# 6. 저장
south_df.to_csv("남서울본부_정렬.csv", index=False, encoding='cp949')
seoul_df.to_csv("서울본부_정렬.csv", index=False, encoding='cp949')

print("✅ 남서울 / 서울 각각 따로 정렬 완료")

✅ 남서울 / 서울 각각 따로 정렬 완료


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

power_df = pd.read_csv("서울본부_정렬.csv", encoding="cp949")
weather_df = pd.read_csv("../project_daily/seoul_2023_2024_final.csv")

power_df["기준일자"] = pd.to_datetime(power_df["기준일자"])
power_df["기준시"] = power_df["기준시"].astype(int)

weather_df["날짜"] = pd.to_datetime(weather_df["날짜"])
weather_df["시간"] = weather_df["시간"].astype(int)

df = pd.merge(
    power_df, weather_df, left_on=["기준일자", "기준시"], right_on=["날짜","시간"], how="inner"
)
print("병합 후 데이터 크기:", df.shape)

df["hour"] = df["기준시"]
df["dayofweek"] = df["기준일자"].dt.dayofweek
df["month"] = df["기준일자"].dt.month

df['CDD'] = (df['기온'] - 24).clip(lower=0)
df['HDD'] = (18 - df['기온']).clip(lower=0)

features = ['기온', '습도', '일사량', 'hour', 'dayofweek', 'month', 'CDD', 'HDD']
target = '전력사용량(MWh)'

x = df[features]
y = df[target]

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, shuffle=False
)

lr = LinearRegression()
lr.fit(x_train, y_train)
y

ModuleNotFoundError: No module named 'sklearn'

In [1]:
import os
print(os.getcwd())
print(os.listdir())

c:\Users\jjwan\Documents\RE_5th\project\filter
['(개방용) 지역본부별 시간대별 전력사용량_20241231.csv', 'filter.ipynb', '남서울본부_정렬.csv', '서울_남서울_본부만.csv', '서울_남서울_정제_최신아래.csv', '서울본부_정렬.csv']


In [4]:
import pandas as pd
weather_df = pd.read_csv("../../project_daily/seoul_2023_2024_final.csv")
print(weather_df.head())

   지점번호 지점명          일시  일조합(hr)  일조율(%)  일사합(MJ/m2)  평균기온(℃)  최고기온(℃) 최고기온시각  \
0   108  서울  2023-01-01      9.0    94.7       10.81     -0.2      3.8  13:43   
1   108  서울  2023-01-02      9.1    95.8       11.63     -4.5     -0.4  15:25   
2   108  서울  2023-01-03      9.1    95.8       11.77     -5.0      0.6  15:55   
3   108  서울  2023-01-04      8.7    91.6       10.89     -1.8      3.3  15:10   
4   108  서울  2023-01-05      4.7    48.5        6.09     -1.6      3.6  15:36   

   최저기온(℃) 최저기온시각일교차  Unnamed: 8  평균습도(%rh)  최저습도(%rh)  
0     -4.3     23:50         8.1       55.0       28.2  
1     -7.4      8:04         7.0       46.0       31.9  
2     -9.0      5:51         9.6       49.0       27.0  
3     -5.7      2:29         9.0       51.0       33.7  
4     -5.6      7:49         9.2       58.1       39.0  
